In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [17]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [4]:
chain=RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x:x['num']*2),  # Will return a Dictionary within a dictionary
    modified=lambda x : x['num']*2  # Will return a Key-Value Pair                          
)

In [5]:
chain.invoke(input={"num":3})

{'extra': {'num': 3, 'mult': 6}, 'modified': 6}

<h3> Application in Streaming </h3>

In [8]:
vectorStore=FAISS.from_texts(
    texts=["Ritish has a 10 month old son !",
           "Rohit is Ritish's Younger Brother",
           "Eklavya is Ritish's Son"
           "Rekha is the name of Ritish's Mother",
           "Anirban is Ritish's Father"],
    embedding=SentenceTransformerEmbeddings())

C:\Users\MSI\AppData\Local\Temp\ipykernel_21660\789142276.py:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=SentenceTransformerEmbeddings())


In [9]:
question={"question":"Who is Anirban's Wife and who is Eklavya's Uncle?"}

In [11]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
            HumanMessagePromptTemplate.from_template
                (
                    template=
                    """
                        Answer the question based only on the following context:
                        {context}
                
                        Question: {question}
                    """
                )
            ],
    
)

In [18]:
retrievalChain=RunnableParallel({"context":vectorStore.as_retriever(k=8),"question":RunnablePassthrough()})|RunnablePassthrough.assign(output=prompt|llm|StrOutputParser())

In [19]:
response=retrievalChain.invoke(input=question['question'])

In [20]:
response

{'context': [Document(id='4a5091e2-b97a-4b6a-aacb-7816b46a9903', metadata={}, page_content="Anirban is Ritish's Father"),
  Document(id='c1f1c7da-5a67-4fca-bb37-8f86f529c908', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"),
  Document(id='829aef03-1808-4c4c-bb9f-9ec0465a9365', metadata={}, page_content="Rohit is Ritish's Younger Brother"),
  Document(id='8ccca5f6-5a9b-4a19-809e-02c000da781f', metadata={}, page_content='Ritish has a 10 month old son !')],
 'question': "Who is Anirban's Wife and who is Eklavya's Uncle?",
 'output': "Based on the provided text:\n\n* **Anirban's Wife:**  The text states Anirban is Ritish's father, but it doesn't name Anirban's wife.\n\n* **Eklavya's Uncle:** The text states Eklavya is Ritish's son and Rohit is Ritish's younger brother.  Therefore, Rohit is Eklavya's uncle."}

In [15]:
print(response['output'])

Based on the provided documents, Anirban's wife is not mentioned. Eklavya's uncle is Rohit.


In [21]:
for chunk in retrievalChain.stream(input=question['question']):
    print(chunk)

{'question': "Who is Anirban's Wife and who is Eklavya's Uncle?"}
{'context': [Document(id='4a5091e2-b97a-4b6a-aacb-7816b46a9903', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='c1f1c7da-5a67-4fca-bb37-8f86f529c908', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(id='829aef03-1808-4c4c-bb9f-9ec0465a9365', metadata={}, page_content="Rohit is Ritish's Younger Brother"), Document(id='8ccca5f6-5a9b-4a19-809e-02c000da781f', metadata={}, page_content='Ritish has a 10 month old son !')]}
{'output': 'Based'}
{'output': " on the provided text:\n\n* **Anirban's Wife:**  "}
{'output': "The text states Anirban is Ritish's father, but it doesn"}
{'output': "'t name Anirban's wife.\n\n* **Eklavya's Uncle:** The text states Eklavya is Ritish's son"}
{'output': " and Rohit is Ritish's younger brother.  Therefore, Rohit is Eklavya's uncle."}
